In [5]:
%pip install -q ipywidgets

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, fixed

In [10]:

def space_generator(thickness, resolution):
    return np.linspace(-thickness/2, thickness/2, num=resolution)


def power_law_volume_fraction(thickness, resolution, exp):
    mesh = space_generator(thickness, resolution)
    v_f = []
    for coordinate in mesh:
        v_f.append(
            (.5 + coordinate / thickness) ** exp
        )
    return v_f


def power_law(P_1, P_2, exp, thickness, resolution):
    v_f = power_law_volume_fraction(thickness, resolution, exp)
    property = []
    for volume_fraction in v_f:
        property.append(volume_fraction * (P_1 - P_2) + P_2)
    return property


def sigmoid_law_volume_fraction(exp, thickness, resolution):
    mesh = space_generator(thickness, resolution)
    f_z_1 = []
    f_z_2 = []
    for coordinate in mesh:
        if coordinate >= 0:
            f_z_1.append(
                (1 - .5 * ((thickness / 2 - coordinate) / (thickness / 2)) ** exp)
            )
        else:
            f_z_2.append(
                .5 * ((thickness / 2 + coordinate) / (thickness / 2)) ** exp
            )
    return f_z_1, f_z_2


def sigmoid_law(P_1, P_2, exponent, thickness, resolution):
    f_z_1, f_z_2 = sigmoid_law_volume_fraction(exponent, thickness, resolution)
    mesh = space_generator(thickness, resolution)
    property = []
    cnt = 0
    cnt_2 = 0
    for coordinate in mesh:
        if coordinate > 0:
            property.append(
                (f_z_1[cnt]) * P_1 + (1 - f_z_1[cnt]) * P_2
            )
            cnt += 1
        else:
            property.append(
                f_z_2[cnt_2] * P_1 + (1 - f_z_2[cnt_2]) * P_2
            )
            cnt_2 += 1
    return property


def exponential_law(P_1, P_2, thickness, resolution):
    mesh = space_generator(thickness, resolution)
    property = [P_2 * np.exp(1/thickness * np.log(P_1/P_2) * (coordinate + thickness/2)) for coordinate in mesh]
    return property


def run_and_visualise(P_1=2, P_2=10, exp=2, h=2, res=20):
    x = space_generator(h, res)
    P_law = power_law(P_1, P_2, exp, h, res)
    P_sigmoid = sigmoid_law(P_1, P_2, exp, h, res)
    P_exp = exponential_law(P_1, P_2, h, res)
    fig, ax = plt.subplots()
    plt.plot(x, P_law, label='Power Law')
    plt.plot(x, P_sigmoid, label='Sigmoid')
    plt.plot(x, P_exp, label='Exponential')
    plt.legend()
    ax.set_xlabel('Distance along build direction (m)')
    ax.set_ylabel('Property')
    plt.show()

In [12]:
w=interactive(run_and_visualise, P_1=(0.1, 100), P_2=(0.1, 100), exp=(1, 20), h=(.1, 20), res=(3, 100))
w

interactive(children=(FloatSlider(value=2.0, description='P_1', min=0.1), FloatSlider(value=10.0, description=…